In [1]:
pip install torch


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

In [4]:
pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('../arch/triplets_over_100k.csv')

In [7]:
df.head()

,Unnamed: 0,City1,City1_state,City1_long,City1_lat,City2,City2_state,City2_long,City2_lat,City3,City3_state,City3_long,City3_lat
0,0,Fort Wayne,IN,-85.1437,41.0886,Denver,CO,-104.8759,39.7621,New Haven,CT,-72.9246,41.3112
1,1,Buffalo,NY,-78.8487,42.9016,Youngstown,OH,-80.6463,41.0993,Mobile,AL,-88.1162,30.6783
2,2,Ogden,UT,-111.9682,41.2278,Boise,ID,-116.2312,43.6007,Columbus,OH,-82.9850,39.9862
3,3,Omaha,NE,-96.0522,41.2627,Los Angeles,CA,-118.4068,34.1139,Kissimmee,FL,-81.4164,28.3042
4,4,Cleveland,OH,-81.6804,41.4767,Fort Wayne,IN,-85.1437,41.0886,Phoenix,AZ,-112.0891,33.5722


In [8]:
pip install geopy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
from geopy.distance import geodesic

def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

df['1-2_distance'] = df.apply(lambda row: calculate_distance(row['City1_lat'], row['City1_long'], row['City2_lat'], row['City2_long']), axis=1)
df['1-3_distance'] = df.apply(lambda row: calculate_distance(row['City1_lat'], row['City1_long'], row['City3_lat'], row['City3_long']), axis=1)
df['1-2closer'] = df.apply(lambda row: 0 if row['1-2_distance'] < row['1-3_distance'] else 1, axis=1)

In [10]:
df.head()

,Unnamed: 0,City1,City1_state,City1_long,City1_lat,City2,City2_state,City2_long,City2_lat,City3,City3_state,City3_long,City3_lat,1-2_distance,1-3_distance,1-2closer
0,0,Fort Wayne,IN,-85.1437,41.0886,Denver,CO,-104.8759,39.7621,New Haven,CT,-72.9246,41.3112,1677.366238,1024.395446,1
1,1,Buffalo,NY,-78.8487,42.9016,Youngstown,OH,-80.6463,41.0993,Mobile,AL,-88.1162,30.6783,249.496461,1586.427248,0
2,2,Ogden,UT,-111.9682,41.2278,Boise,ID,-116.2312,43.6007,Columbus,OH,-82.9850,39.9862,438.765496,2445.438893,0
3,3,Omaha,NE,-96.0522,41.2627,Los Angeles,CA,-118.4068,34.1139,Kissimmee,FL,-81.4164,28.3042,2117.696059,1959.381674,1
4,4,Cleveland,OH,-81.6804,41.4767,Fort Wayne,IN,-85.1437,41.0886,Phoenix,AZ,-112.0891,33.5722,293.300314,2810.596064,0


In [11]:
prompt = pd.read_csv('../prompts/over_100k_prompts.csv')

In [12]:
prompt.head()

,Unnamed: 0,Prompt
0,0,"Is Fort Wayne, IN closer to Denver, CO or New ..."
1,1,"Is Buffalo, NY closer to Youngstown, OH or Mob..."
2,2,"Is Ogden, UT closer to Boise, ID or Columbus, OH?"
3,3,"Is Omaha, NE closer to Los Angeles, CA or Kiss..."
4,4,"Is Cleveland, OH closer to Fort Wayne, IN or P..."


In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
df['prompt_tokenized'] = prompt['Prompt'].apply(lambda x: tokenizer.tokenize(x))

In [14]:
df.head()

,Unnamed: 0,City1,City1_state,City1_long,City1_lat,City2,City2_state,City2_long,City2_lat,City3,City3_state,City3_long,City3_lat,1-2_distance,1-3_distance,1-2closer,prompt_tokenized
0,0,Fort Wayne,IN,-85.1437,41.0886,Denver,CO,-104.8759,39.7621,New Haven,CT,-72.9246,41.3112,1677.366238,1024.395446,1,"[is, fort, wayne, ,, in, closer, to, denver, ,..."
1,1,Buffalo,NY,-78.8487,42.9016,Youngstown,OH,-80.6463,41.0993,Mobile,AL,-88.1162,30.6783,249.496461,1586.427248,0,"[is, buffalo, ,, ny, closer, to, young, ##stow..."
2,2,Ogden,UT,-111.9682,41.2278,Boise,ID,-116.2312,43.6007,Columbus,OH,-82.9850,39.9862,438.765496,2445.438893,0,"[is, ogden, ,, ut, closer, to, boise, ,, id, o..."
3,3,Omaha,NE,-96.0522,41.2627,Los Angeles,CA,-118.4068,34.1139,Kissimmee,FL,-81.4164,28.3042,2117.696059,1959.381674,1,"[is, omaha, ,, ne, closer, to, los, angeles, ,..."
4,4,Cleveland,OH,-81.6804,41.4767,Fort Wayne,IN,-85.1437,41.0886,Phoenix,AZ,-112.0891,33.5722,293.300314,2810.596064,0,"[is, cleveland, ,, oh, closer, to, fort, wayne..."


In [15]:
df.to_csv('BERT_data.csv')

In [16]:
df = pd.read_csv('BERT_data.csv')
data = df[['City1', 'City1_state', 'City2', 'City2_state', '1-2_distance']]


In [17]:
data['City1'] = data['City1'] + ', ' + data['City1_state']
data['City2'] = data['City2'] + ', ' + data['City2_state']
data.head()

/var/folders/6n/tkyy6kcj11s3n3dzjcx0c09r0000gn/T/ipykernel_28408/442874184.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['City1'] = data['City1'] + ', ' + data['City1_state']
/var/folders/6n/tkyy6kcj11s3n3dzjcx0c09r0000gn/T/ipykernel_28408/442874184.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['City2'] = data['City2'] + ', ' + data['City2_state']


,City1,City1_state,City2,City2_state,1-2_distance
0,"Fort Wayne, IN",IN,"Denver, CO",CO,1677.366238
1,"Buffalo, NY",NY,"Youngstown, OH",OH,249.496461
2,"Ogden, UT",UT,"Boise, ID",ID,438.765496
3,"Omaha, NE",NE,"Los Angeles, CA",CA,2117.696059
4,"Cleveland, OH",OH,"Fort Wayne, IN",IN,293.300314


In [28]:
data

,City1,City2,1-2_distance,City1_tokens,City2_tokens,City1_embedding,City2_embedding
0,"Fort Wayne, IN","Denver, CO",1677.366238,"[101, 3481, 6159, 1010, 1999, 102]","[101, 7573, 1010, 2522, 102]","[-0.31507787, 0.0428125, -0.32437178, -0.13091...","[-0.16230181, 0.042178527, -0.16107842, 0.0330..."
1,"Buffalo, NY","Youngstown, OH",249.496461,"[101, 6901, 1010, 6396, 102]","[101, 2402, 13731, 1010, 2821, 102]","[-0.11049869, -0.090624474, -0.41215602, -0.00...","[-0.5519003, 0.07758211, -0.48017582, 0.053052..."
2,"Ogden, UT","Boise, ID",438.765496,"[101, 23203, 1010, 21183, 102]","[101, 23193, 1010, 8909, 102]","[0.0754463, 0.1570905, -0.25322294, 0.00835996...","[0.27816305, 0.27268773, -0.33497304, 0.023465..."
3,"Omaha, NE","Los Angeles, CA",2117.696059,"[101, 12864, 1010, 11265, 102]","[101, 3050, 3349, 1010, 6187, 102]","[-0.20617945, -0.10607363, -0.4682085, 0.17641...","[0.21018803, 0.07478464, -0.05120385, 0.056763..."
4,"Cleveland, OH","Fort Wayne, IN",293.300314,"[101, 6044, 1010, 2821, 102]","[101, 3481, 6159, 1010, 1999, 102]","[-0.39888722, 0.2881037, -0.26733533, -0.11915...","[-0.31507787, 0.0428125, -0.32437178, -0.13091..."
...,...,...,...,...,...,...,...
155,"Lancaster, PA","Philadelphia, PA",99.716497,"[101, 10237, 1010, 6643, 102]","[101, 4407, 1010, 6643, 102]","[-0.2404221, -0.15628585, -0.34284556, 0.20818...","[-0.07053614, -0.103911854, -0.3210321, 0.0200..."
156,"Syracuse, NY","Indianapolis, IN",910.701455,"[101, 11736, 1010, 6396, 102]","[101, 9506, 1010, 1999, 102]","[-0.13082203, -0.009349303, -0.34579292, -0.00...","[-0.42447335, -0.0046665245, -0.27134463, -0.0..."
157,"Philadelphia, PA","Durham, NC",556.235844,"[101, 4407, 1010, 6643, 102]","[101, 9296, 1010, 13316, 102]","[-0.07053614, -0.103911854, -0.3210321, 0.0200...","[-0.12904654, -0.057629235, -0.086578354, 0.08..."
158,"Anaheim, CA","Round Lake Beach, IL",2760.949226,"[101, 20835, 1010, 6187, 102]","[101, 2461, 2697, 3509, 1010, 6335, 102]","[0.059194528, 0.017007602, -0.1521374, 0.15859...","[0.006000459, 0.03072274, -0.14728066, 0.33253..."


In [18]:
data = data.drop(['City1_state', 'City2_state'], axis=1)
data.head()

,City1,City2,1-2_distance
0,"Fort Wayne, IN","Denver, CO",1677.366238
1,"Buffalo, NY","Youngstown, OH",249.496461
2,"Ogden, UT","Boise, ID",438.765496
3,"Omaha, NE","Los Angeles, CA",2117.696059
4,"Cleveland, OH","Fort Wayne, IN",293.300314


In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
data['City1_tokens'] = data['City1'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
data['City2_tokens'] = data['City2'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

In [20]:
pip install torch torchvision


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
import torch, torchvision
x = torch.rand(5, 3)
print(x)


tensor([[0.0689, 0.1581, 0.7658],
        [0.9579, 0.3599, 0.6362],
        [0.7995, 0.6915, 0.9611],
        [0.2083, 0.4963, 0.4836],
        [0.1039, 0.6048, 0.2887]])


In [22]:
model = BertModel.from_pretrained('bert-base-uncased')
def get_embedding(tokens):
    with torch.no_grad():
        outputs = model(torch.tensor(tokens).unsqueeze(0))
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding
data['City1_embedding'] = data['City1_tokens'].apply(get_embedding)
data['City2_embedding'] = data['City2_tokens'].apply(get_embedding)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [23]:
from torch import nn, optim

class SimpleClassifier(nn.Module):
    def __init__(self):
        super(SimpleClassifier, self).__init__()
        self.linear = nn.Linear(768*3, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

model = SimpleClassifier()
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [24]:
data.to_csv("BERT_data.csv")

In [27]:
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

combined_embeddings = torch.cat((data['City1_embedding'], data['City2_embedding']), dim=1)

distances = data['1-2_distance'].values
distances = torch.tensor(distances).float()

# Split data
X_train, X_test, y_train, y_test = train_test_split(combined_embeddings, distances, test_size=0.2)

class DistancePredictor(nn.Module):
    def __init__(self):
        super(DistancePredictor, self).__init__()
        self.fc1 = nn.Linear(in_features=768*2, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = DistancePredictor()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

# Train
for epoch in range(8):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # fp
        predictions = model(data)
        loss = criterion(predictions.squeeze(), targets)

        # bp
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

TypeError: expected Tensor as element 0 in argument 0, but got Series